In [ ]:
%pip install -U langchain-community

In [ ]:
%pip install langchain-core

In [ ]:
%pip install langchain-openai

In [ ]:
# Step 1: Import required libraries
import os
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.docstore.document import Document
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# Configuration file to manage API KEY as environment variable
# Install: pip install python-dotenv
from dotenv import load_dotenv
import os
# Load API KEY information
load_dotenv(override=True)

In [ ]:
import os
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [ ]:

import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

from langchain.chat_models import init_chat_model

# Initialize your chat model using the specified model provider
model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Spanish"),
    HumanMessage("hi!"),
]

model.invoke(messages)

# Test with Open AI
Change the invoke string

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Tell me a joke about graduate students.")

AIMessage(content='Why did the AI go broke?\n\nBecause it had too many bugs in its system!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 14, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BCF84rflvBI00RwpLf2xC7MxCfiu0', 'finish_reason': 'stop', 'logprobs': None}, id='run-fd3bebff-8cbc-4e28-bbec-97e91c9be159-0', usage_metadata={'input_tokens': 14, 'output_tokens': 18, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})